#### **VAKUL GUPTA - 170101076**
# **INTELLIGENT SYSTEMS AND INTERFACES - CS565 - ASSIGNMENT 2**

### 0 - PREREQUISITE

##### Mount google drive

In [ ]:
# mount google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##### Install basic libraries

In [ ]:
# install basic libraries
import random
import nltk
import codecs
import math
import statistics
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

##### Read and load english dataset

In [ ]:
# read and load english corpus
englishText = codecs.open('/content/drive/My Drive/Intelligent_Systems_CS565/Assignment-2/en_wiki.txt', 'r').read()

# filter corpus to take a percantage of it
filterRatio = 0.1
englishText = englishText[0:int(filterRatio*len(englishText))]

##### Sentence Segmentation

In [ ]:
# sentence segmentation
listOfEnglishSentences = []
listOfSentences = nltk.sent_tokenize(englishText)
for sentence in listOfSentences:
    listOfEnglishSentences.append(sentence)
print(len(listOfEnglishSentences))

761582


##### Word Tokenization

In [ ]:
# function to convert a sentence into a list of words
def performWordTokenisation(sentences):
    wordTokens = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        for word in words:
            wordTokens.append(word)
    
    return wordTokens

##### Split Data into training, devlopment and test sets

In [ ]:
# split data into part-1 (90%) and part-2 (10%)
numberOfTokens = len(listOfEnglishSentences)
# randomly shuffle sentences for splitting
random.shuffle(listOfEnglishSentences)
datasetTrainAndDevelopment = listOfEnglishSentences[:int((numberOfTokens+1)*.9)]
part2 = listOfEnglishSentences[int((numberOfTokens+1)*.9):]
# fixed test set
datasetTest = performWordTokenisation(part2)

##### Utility Function for splitting part-1 data into training and development sets

In [ ]:
# given part-1 split randomly into training set (90%) and development set (10%)
def getTrainingAndDevelopmentSet(datasetTrainAndDevelopment):
    random.shuffle(datasetTrainAndDevelopment)
    datasetTrain = datasetTrainAndDevelopment[: int( (len(datasetTrainAndDevelopment)+1)*.9)]
    datasetDevelopment = datasetTrainAndDevelopment[int( (len(datasetTrainAndDevelopment)+1)*.9):]
    datasetTrain=performWordTokenisation(datasetTrain)
    datasetDevelopment=performWordTokenisation(datasetDevelopment)
    return datasetTrain, datasetDevelopment

### 1 - N-GRAM LANGUAGE MODEL

#### 1.1 Implement Discounting and Interpolation Smoothing Methods

###### Utility functions for obtaining Ngram unsmoothed probabilities

In [ ]:
# get unsmoothed probability for unigrams using prob = frequency of a particular unigram / total
def getProbUnigramDict(freqOfUnigramDict):
    totalNumberOfTokens = 0
    for item in freqOfUnigramDict:
        totalNumberOfTokens += freqOfUnigramDict[item]
    probUnigramDict = {}
    for unigram in freqOfUnigramDict:
        probUnigramDict[unigram] = (freqOfUnigramDict.get(unigram)) / totalNumberOfTokens
    return probUnigramDict

# get unsmoothed probability for bigrams using prob = frequency of a particular bigram / total
def getProbBigramDict(allBigramsList, freqOfUnigramDict, freqOfBigramDict):
    probBigramDict = {}
    for bigram in allBigramsList:
        probBigramDict[bigram] = (freqOfBigramDict.get(bigram)) / (freqOfUnigramDict.get(bigram[0]))
    return probBigramDict

# get unsmoothed probability for trigrams using prob = frequency of a particular trigram / total
def getProbTrigramDict(allTrigramsList, freqOfBigramDict, freqOfTrigramDict):
    probTrigramDict = {}
    for trigram in allTrigramsList:
        probTrigramDict[trigram] = (freqOfTrigramDict.get(trigram)) / (freqOfBigramDict.get((trigram[0], trigram[1])))
    return probTrigramDict

###### Utility function for obtaining Ngram list and frequencies

In [ ]:
# get all the details of the dataset including -
# list of all bigrams and trigrams
# dictionaries containing frequency of unique unigrams, bigrams and trigrams
def getFreqAndListNgrams(datasetTrain):
    # frequency dictionaries of N-grams
    freqOfBigramDict = {}
    freqOfUnigramDict = {}
    freqOfTrigramDict = {}

    # list of all bigrams and trigrams
    allBigramsList = []
    allTrigramsList = []

    for i in range(len(datasetTrain)):
        if i < len(datasetTrain) - 1:
            allBigramsList.append((datasetTrain[i], datasetTrain[i + 1]))
            if (datasetTrain[i], datasetTrain[i + 1]) in freqOfBigramDict:
                freqOfBigramDict[(datasetTrain[i], datasetTrain[i + 1])] += 1
            else:
                freqOfBigramDict[(datasetTrain[i], datasetTrain[i + 1])] = 1

            if i < len(datasetTrain) - 2:
                allTrigramsList.append((datasetTrain[i], datasetTrain[i + 1], datasetTrain[i + 2]))
                if (datasetTrain[i], datasetTrain[i + 1], datasetTrain[i + 2]) in freqOfTrigramDict:
                    freqOfTrigramDict[(datasetTrain[i], datasetTrain[i + 1], datasetTrain[i + 2])] += 1
                else:
                    freqOfTrigramDict[(datasetTrain[i], datasetTrain[i + 1], datasetTrain[i + 2])] = 1

        if datasetTrain[i] in freqOfUnigramDict:
            freqOfUnigramDict[datasetTrain[i]] += 1
        else:
            freqOfUnigramDict[datasetTrain[i]] = 1

    return freqOfUnigramDict, freqOfBigramDict, freqOfTrigramDict, allBigramsList, allTrigramsList

###### Utility function for getting all Ngram Details

In [ ]:
# function for obtaining probabilities, and frequency distributions of all unigrams, bigrams and trigrams of a particular dataset
def getAlldatasetTrainDetails(datasetTrain):
    freqOfUnigramDict, freqOfBigramDict, freqOfTrigramDict, allBigramsList, allTrigramsList = getFreqAndListNgrams(datasetTrain)
    probUnigramDict = getProbUnigramDict(freqOfUnigramDict)
    probBigramDict = getProbBigramDict(allBigramsList, freqOfUnigramDict, freqOfBigramDict)
    probTrigramDict = getProbTrigramDict(allTrigramsList, freqOfBigramDict, freqOfTrigramDict)
    return freqOfUnigramDict, freqOfBigramDict, freqOfTrigramDict, allBigramsList, allTrigramsList, probUnigramDict, probBigramDict, probTrigramDict

###### DISCOUNTING IMPLEMENTATION

###### Utility function for obtaining trigram discounted smoothed probabilities

In [ ]:
# function for calculating probabilities of trigrams after disconting
def getDiscountedProbTrigramDict(allTrigramsList, freqOfTrigramDict, freqOfBigramDict, beta):
    discountedProbTrigramDict = {}
    for trigram in allTrigramsList:
        # the corresponding count is decreased by beta amount
        count = (freqOfTrigramDict.get(trigram) - beta)
        if count < 0:
            count = 0
        discountedProbTrigramDict[trigram] = count / freqOfBigramDict.get((trigram[0], trigram[1]))
    
    return discountedProbTrigramDict

###### Utility function for obtaining perplexity after discounting method

In [ ]:
# calculates perplexity (evaluation performance) on test data after discounting method
def evaluatePerplexityDiscounting(datasetTest, freqOfTrigramDict, discountedProbTrigramDict):
    total = 0
    # summation of logarithm of discounted probabilities
    sumOfLogProb = 0
    totalTrigrams = 0
    for item in freqOfTrigramDict:
        totalTrigrams += freqOfTrigramDict[item]
    for j in range(len(datasetTest) - 2):
        trigram = (datasetTest[j], datasetTest[j + 1], datasetTest[j + 2])
        if trigram in discountedProbTrigramDict:
            prob = discountedProbTrigramDict[trigram]
        else:
            prob = freqOfTrigramDict[trigram] / totalTrigrams

        if prob == 0:
            sumOfLogProb += 0
        else:
            sumOfLogProb += math.log(prob, 2)
        total += 1
    # calculates the entropy
    entropy = (-1 * sumOfLogProb ) / total
    # perplexity is 2^entropy
    perplexity = math.pow(2, entropy)
    return perplexity

###### Utility function for finding optimal parameter and corresponding perplexity

In [ ]:
# finds the log likelihood value for a particular beta
def getLogLikelihoodVal(beta, datasetTrain, datasetTest):
    freqOfUnigramDict, freqOfBigramDict, freqOfTrigramDict, allBigramsList, allTrigramsList, probUnigramDict, probBigramDict, probTrigramDict = getAllTrainDataDetails(datasetTrain)
    freqOfUnigramDictTest, freqOfBigramDictTest, freqOfTrigramDictTest, allBigramsListTest, allTrigramsListTest = getFreqAndListNgrams(datasetTest)

    curLikelihood = 0
    tokens = 0
    for trigram, freq in freqOfTrigramDict.items():
        curProb = probTrigramDict[trigram]
        tokens += freq

        if curProb != 0:
            curLikelihood += (freq * math.log(curProb, 2))
    
    return curLikelihood

# find the optimal parameter by maximizing the log likelihood test 
def findOptimalBeta(datasetTrain, datasetTest):
    beta = 0.1
    optimalBeta = 0
    bestVal = 0
    while beta <= 1:
        tempVal = getLogLikelihoodVal(beta, datasetTrain, datasetTest)

        if tempVal > bestVal:
            bestVal = tempVal
            optimalBeta = beta
        beta += 0.1

    return optimalBeta

# find optimal beta and calculate perplexity after discounting for that optimal parameter
def findOptimalBetaAndPerplexityDiscounting(datasetTrain, datasetTest):
    freqOfUnigramDict, freqOfBigramDict, freqOfTrigramDict, allBigramsList, allTrigramsList, probUnigramDict, probBigramDict, probTrigramDict = getAlldatasetTrainDetails(datasetTrain)
    freqOfUnigramDictTest, freqOfBigramDictTest, freqOfTrigramDictTest, allBigramsListTest, allTrigramsListTest = getFreqAndListNgrams(datasetTest)

    optimalBeta = findOptimalBeta(datasetTrain,datasetTest)
    discountedProbTrigramDict = getDiscountedProbTrigramDict(allTrigramsList, freqOfTrigramDict, freqOfBigramDict, optimalBeta)
    perplexity = evaluatePerplexityDiscounting(datasetTest, freqOfTrigramDictTest, discountedProbTrigramDict)

    return optimalBeta, perplexity

###### INTERPOLATION IMPLEMENTATION

###### Utility function for finding optimal parameters

In [ ]:
# finds the optimal parameter set (lambda1,lambda2,lambda3)
def findOptimalParameters(tokens, epsilon, probUnigramDict, probBigramDict, probTrigramDict):
    parameterSet = [0.2, 0.3, 0.5]
    # repeatedly perform a certain process
    while 1:
        countExpected = [0, 0, 0]
        for j in range(len(tokens) - 2):
            trigram = (tokens[j], tokens[j + 1], tokens[j + 2])
            bigram = (tokens[j], tokens[j + 1])
            tProb = probTrigramDict.get(trigram, 0)
            bProb = probBigramDict.get(bigram, 0)
            uProb = probUnigramDict.get(tokens[j], 0)
            weightedProb = parameterSet[0] * uProb + parameterSet[1] * bProb + parameterSet[2] * tProb
            if weightedProb > 0:
                countExpected[0] += (parameterSet[0] * uProb) / weightedProb
                countExpected[1] += (parameterSet[1] * bProb) / weightedProb
                countExpected[2] += (parameterSet[2] * tProb) / weightedProb
        
        newParameterSet = [0,0,0]
        sumTotal = countExpected[0] + countExpected[1] + countExpected[2] 
        if sumTotal == 0:
            return parameterSet

        # finds the new parameter set using expected counts
        newParameterSet[0] = countExpected[0] / sumTotal
        newParameterSet[1] = countExpected[1] / sumTotal
        newParameterSet[2] = countExpected[2] / sumTotal

        # check if all the three parameters differs less than epsilon from the current parameter set 
        isSaturated = True
        for i in range(0,3):
            diff = abs( newParameterSet[i] - parameterSet[i] )
            if diff > epsilon:
                isSaturated = False
                break
        if isSaturated == True:
            return parameterSet
        else:
            # if not saturated, update the parameter set and repeat the process again
            parameterSet = newParameterSet

###### Utility function for obtaining perplexity after interpolation method

In [ ]:
# calculates the perplexity after interpolation method
def evaluatePerplexityInterpolation(tokens, parameterSet, probUnigramDict, probBigramDict, probTrigramDict):
    # summation of logarithm of discounted probabilities
    sumOfLogProb = 0
    for j in range(len(tokens) - 2):
        trigram = (tokens[j], tokens[j + 1], tokens[j + 2])
        bigram = (tokens[j], tokens[j + 1])
        tProb = probTrigramDict.get(trigram, 0)
        bProb = probBigramDict.get(bigram, 0)
        uProb = probUnigramDict.get(tokens[j], 0)
        weightedProb = parameterSet[0] * uProb + parameterSet[1] * bProb + parameterSet[2] * tProb
        if weightedProb == 0:
            sumOfLogProb += 0
        else:
            sumOfLogProb += math.log(weightedProb, 2)
    totalNumberOfTrigrams = len(tokens) - 2;
    # calculates the entropy
    entropy = (-1 * sumOfLogProb ) / totalNumberOfTrigrams
    # perplexity is 2^entropy
    perplexity = math.pow(2, entropy)
    return perplexity

###### Utility function for finding optimal parameters and corresponding perplexity

In [ ]:
# get optimal parameters and the perplexity value on those parameters
def getLambdaAndPerplexityInterpolation(datasetTrain, datasetTest):
    freqOfUnigramDict, freqOfBigramDict, freqOfTrigramDict, allBigramsList, allTrigramsList, probUnigramDict, probBigramDict, probTrigramDict = getAlldatasetTrainDetails(datasetTrain)
    parameterSet = findOptimalParameters(datasetTest, 0.005, probUnigramDict, probBigramDict, probTrigramDict)
    perplexity = evaluatePerplexityInterpolation(datasetTest, parameterSet, probUnigramDict, probBigramDict, probTrigramDict)
    return parameterSet, perplexity

#### 1.2 Split Part-1 Data

In [ ]:
# split the part-1 data into five set of training and development data randomly shuffled
datasetTrainList = []
datasetDevelopmentList = []
for iteration in range(0,5):
    datasetTrain, datasetDevelopment = getTrainingAndDevelopmentSet(datasetTrainAndDevelopment)
    print('Dataset Train',iteration+1)
    print(datasetTrain[0:5])
    print('Dataset Development',iteration+1)
    print(datasetDevelopment[0:5])
    datasetTrainList.append(datasetTrain)
    datasetDevelopmentList.append(datasetDevelopment)

Dataset Train 1
['Males', 'had', 'a', 'median', 'income']
Dataset Development 1
['``', 'The', 'World', 'Tonight', "''"]
Dataset Train 2
['A', '``', 'Draft', 'Eisenhower', "''"]
Dataset Development 2
['With', 'the', 'exception', 'of', '2009']
Dataset Train 3
['Females', 'have', 'two', 'of', 'the']
Dataset Development 3
['Primary', 'and', 'secondary', 'education', 'is']
Dataset Train 4
['As', 'the', 'fears', 'of', 'a']
Dataset Development 4
['The', 'woman', ',', 'a', 'journalist']
Dataset Train 5
['In', 'the', 'Shiva', 'Purana', ',']
Dataset Development 5
['Not', 'only', 'the', 'first', 'use']


#### 1.3 Model's Performance on Development Set

###### Optimal Parameters and Perplexity - *Interpolation*

In [ ]:
# finds the optimal parameter set and the corresponding perplexity on five different development sets for interpolation
for iteration in range(0,5):
    parameterSet, perplexityInterpolation = getLambdaAndPerplexityInterpolation(datasetTrainList[iteration], datasetDevelopmentList[iteration])
    print('Iteration: ',iteration+1)
    print('Optimal paramater set: ',parameterSet)
    print('Perplexity Interpolation: ', perplexityInterpolation)
    print('\n')

Iteration:  1
Optimal paramater set:  [0.32164648269469676, 0.4108400100831291, 0.26751350722217404]
Perplexity Interpolation:  83.33429776721303


Iteration:  2
Optimal paramater set:  [0.3218061078363997, 0.41100600387609304, 0.2671878882875072]
Perplexity Interpolation:  83.47264677065282


Iteration:  3
Optimal paramater set:  [0.3223139417159694, 0.4097407533483232, 0.26794530493570745]
Perplexity Interpolation:  83.51132068676277


Iteration:  4
Optimal paramater set:  [0.3213984794067617, 0.41077001648718775, 0.26783150410605056]
Perplexity Interpolation:  83.10668348051617


Iteration:  5
Optimal paramater set:  [0.3209134531994651, 0.41041508808930366, 0.26867145871123127]
Perplexity Interpolation:  82.65508917158509




###### Optimal Parameters and Perplexity - *Discounting*

In [ ]:
# finds the optimal beta and the corresponding perplexity on five different development sets for discounting
for iteration in range(0,5):
    beta, perplexityDiscounting = findOptimalBetaAndPerplexityDiscounting(datasetTrainList[iteration], datasetDevelopmentList[iteration])
    print('Iteration: ',iteration+1)
    print('Optimal paramater set: ',beta)
    print('Perplexity Discounting: ', perplexityDiscounting)
    print('\n')

Iteration:  1
Optimal paramater set:  0.67
Perplexity Discounting:  402.02378918743557


Iteration:  2
Optimal paramater set:  0.62
Perplexity Discounting:  388.79809816818016


Iteration:  3
Optimal paramater set:  0.6
Perplexity Discounting:  383.4649507845885


Iteration:  4
Optimal paramater set:  0.64
Perplexity Discounting:  368.53634401389087


Iteration:  5
Optimal paramater set:  0.67
Perplexity Discounting:  376.5628163719391




#### 1.4 Model's Performance on Test Set

###### Optimal Parameters and Perplexity - *Interpolation*

In [ ]:
# finds the optimal parameter set and the corresponding perplexity on test set for interpolation trained on five different sets
listOfperplexityInterpolation = []
for iteration in range(0,5):
    parameterSet, perplexityInterpolation = getLambdaAndPerplexityInterpolation(datasetTrainList[iteration], datasetTest)
    listOfperplexityInterpolation.append(perplexityInterpolation)
    print('Iteration: ',iteration+1)
    print('Optimal paramater set: ',parameterSet)
    print('Perplexity Interpolation: ', perplexityInterpolation)
    print('\n')

Iteration:  1
Optimal paramater set:  [0.3211033830089687, 0.4095029299449679, 0.2693936870460634]
Perplexity Interpolation:  82.8701656214495


Iteration:  2
Optimal paramater set:  [0.3211017789074348, 0.409491066321625, 0.2694071547709402]
Perplexity Interpolation:  82.85584296951804


Iteration:  3
Optimal paramater set:  [0.3211910403132751, 0.4094774173354341, 0.26933154235129086]
Perplexity Interpolation:  82.87979338483134


Iteration:  4
Optimal paramater set:  [0.3211519344538609, 0.40959752882889033, 0.2692505367172487]
Perplexity Interpolation:  82.91541109966278


Iteration:  5
Optimal paramater set:  [0.32098491142145213, 0.4094641674474203, 0.26955092113112755]
Perplexity Interpolation:  82.82966927039946




###### Optimal Parameters and Perplexity - *Discounting*

In [ ]:
# finds the optimal beta and the corresponding perplexity on test set for discounting trained on five different sets
listOfperplexityDiscounting = []
for iteration in range(0,5):
    beta, perplexityDiscounting = findOptimalBetaAndPerplexityDiscounting(datasetTrainList[iteration], datasetTest)
    listOfperplexityDiscounting.append(perplexityDiscounting)
    print('Iteration: ',iteration+1)
    print('Optimal paramater set: ',beta)
    print('Perplexity Discounting: ', perplexityDiscounting)
    print('\n')

Iteration:  1
Optimal paramater set:  0.67
Perplexity Discounting:  372.22469249797877


Iteration:  2
Optimal paramater set:  0.65
Perplexity Discounting:  371.58097868999346


Iteration:  3
Optimal paramater set:  0.69
Perplexity Discounting:  377.72073464050703


Iteration:  4
Optimal paramater set:  0.68
Perplexity Discounting:  376.5780896216441


Iteration:  5
Optimal paramater set:  0.67
Perplexity Discounting:  374.9597414209729




#### 1.5 Laplace Smoothing

###### Utility function for obtaining trigram laplace smoothed probabilities

In [ ]:
# calculates the smoothed probabilities for trigrams after laplace smoothing
def getLaplaceProbTrigramDict(freqOfTrigramDict, freqOfBigramDict):
    K = 1
    laplaceProbTrigramDict = {}
    for trigram in freqOfTrigramDict:
        bigram = (trigram[0], trigram[1])
        laplaceProbTrigramDict[trigram] = (freqOfTrigramDict.get(trigram) + K) / (freqOfBigramDict.get(bigram) + K * len(freqOfTrigramDict))
    return laplaceProbTrigramDict

###### Utility function for obtaining perplexity after laplace smoothing

In [ ]:
# calculates the perplexity after laplace smoothing
def evaluatePerplexityLaplace(tokens, laplaceProbTrigramDict):
    total = 0
    # summation of logarithm of discounted probabilities
    sumOfLogProb = 0
    for j in range(len(tokens) - 2):
        trigram = (tokens[j], tokens[j + 1], tokens[j + 2])
        if trigram in laplaceProbTrigramDict:
            prob = laplaceProbTrigramDict[trigram]
        else:
            prob = 0
        if prob == 0:
            sumOfLogProb += 0
        else:
            sumOfLogProb += math.log(prob, 2)
        total += 1
    # calculates the entropy
    entropy = (-1 * sumOfLogProb ) / total
    # perplexity is 2^entropy
    perplexity = math.pow(2, entropy)
    return perplexity

In [ ]:
# utility function for obtaining perplexity
def getPerplexityLaplace(datasetTrain, datasetTest):
    freqOfUnigramDict, freqOfBigramDict, freqOfTrigramDict, allBigramsList, allTrigramsList, probUnigramDict, probBigramDict, probTrigramDict = getAlldatasetTrainDetails(datasetTrain)
    laplaceProbTrigramDict = getLaplaceProbTrigramDict(freqOfTrigramDict, freqOfBigramDict)
    perplexity = evaluatePerplexityLaplace(datasetTest, laplaceProbTrigramDict)
    return perplexity

In [ ]:
# finds the perplexity values on test set for laplace smoothing trained on five different sets
listOfperplexityLaplace = []
for iteration in range(0,5):
    perplexityLaplace = getPerplexityLaplace(datasetTrainList[iteration], datasetTest)
    listOfperplexityLaplace.append(perplexityLaplace)
    print('Iteration: ',iteration+1)
    print('Perplexity Laplace: ', perplexityLaplace)
    print('\n')

Iteration:  1
Perplexity Laplace:  668.4223883436812


Iteration:  2
Perplexity Laplace:  667.709718953416


Iteration:  3
Perplexity Laplace:  666.1954258895079


Iteration:  4
Perplexity Laplace:  666.4257546927955


Iteration:  5
Perplexity Laplace:  665.7075762986127




###### Variance of Perplexity Values

In [ ]:
# obtain variance of perplexity values after interpolation, discounting and laplace methods
print("Variance of perplexity laplace is % s" %(statistics.variance(listOfperplexityLaplace)))
print("Variance of perplexity interpolation is % s" %(statistics.variance(listOfperplexityInterpolation)))
print("Variance of perplexity discounting is % s" %(statistics.variance(listOfperplexityDiscounting)))

Variance of perplexity laplace is 1.2790530755490683
Variance of perplexity interpolation is 0.00085600000020349
Variance of perplexity discounting is 5.7136960000033


### 2 - Vector Semantics:  GloVE implementation

#### 2.0 - PREREQUISITES

#### Install basic libraries

In [ ]:
# Import required libraries
! rm -rf web
! cp -r '/content/drive/My Drive/Intelligent_Systems_CS565/Assignment-2/word-embeddings-benchmarks/web' .
import numpy as np
from math import log
import pickle
from tqdm import tqdm
from collections import Counter
from scipy import sparse
from web.datasets.similarity import *
from web.evaluate import evaluate_similarity
from web.embeddings import fetch_GloVe

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


#### 2.1 - GloVe embedding method implementation

###### Parameters used for glove embeddings implementation

In [ ]:
# declaring parameters used in glove embeddings implementation
CONTEXT_WINDOW = 10
DIM_SIZE = 100
ITERATIONS = 45
LEARNING_RATE = 0.05
X_MAX = 100
ALPHA = 0.75

###### Utility function for constructing vocabulary

In [ ]:
# Builds a vocabulary of words mapped to word ID and word frequency in the corpus
def constructVocab(dataset):
  vocab = Counter()
  listOfSentences = nltk.sent_tokenize(dataset)
  for sentence in listOfSentences:
    listOfTokens = nltk.word_tokenize(sentence)
    vocab.update(listOfTokens)
  return {token: (tokenID, tokenFrequency) for tokenID, (token, tokenFrequency) in enumerate(vocab.items())}, listOfSentences

###### Utility function for constructing co-occurence matrix

In [ ]:
#Builds the word Co-occurrence matrix X for the given vocabulary.
# This is represented in a list of tuples containt {centerID, contextID, Xij}
def constructCoOccurenceMatrix(vocab, listOfSentences, contextWindow):
  lengthOfVocab = len(vocab)
  output = []
  #initialize a matrix of size V * V where V is the vocabSize
  cooccurrences = sparse.lil_matrix((lengthOfVocab, lengthOfVocab), dtype=np.float64)
  #Iterate over all the sentences in the corpus
  for i, sentence in enumerate(listOfSentences):
    listOfTokens = nltk.word_tokenize(sentence)
    tokenIds = [vocab[token][0] for token in listOfTokens]

    for i, centerId in enumerate(tokenIds):
      contextIds = tokenIds[max(0, i - contextWindow) : i]
      contextLen = len(contextIds)

      #Iterate over each context word in left window
      for j, contextId in enumerate(contextIds):
        distance = contextLen - j
        increment = 1.0 / float(distance)
        #Update the matrix for contextWord and centerWord symmetrically to handle both left and right window co-occurences
        cooccurrences[centerId, contextId] += increment
        cooccurrences[contextId, centerId] += increment
  
  #Build the output in format of list of tuples from the sparse matrix
  for i, (row, data) in enumerate(zip(cooccurrences.rows, cooccurrences.data)):
    for dataIndex, j in enumerate(row):
      output.append((i, j, data[dataIndex]))
  
  return output

###### Utility function for performing a particular iteration

In [ ]:
#Runs a single iteration of AdaGrad while training the GloVe embeddings.
#Returns the cost associated with the given weights and updates the weights by AdaGrad logic
def performIteration(vocab, data, rateOfLearning, xMax, alpha):
  costGlobal = 0
  random.shuffle(data)

  for (vMain, vContext, bMain, bContext, gradWMain, gradWContext,
       gradBMain, gradBContext, cooccurrence) in data:
      
    weight = (cooccurrence / xMax) ** alpha if cooccurrence < xMax else 1
    innerCost = (vMain.dot(vContext) + bMain[0] + bContext[0] - log(cooccurrence))
    cost = weight * (innerCost ** 2)
    costGlobal += 0.5 * cost

    #Compute gradients for word vectors and bias terms
    mainGradient = weight * innerCost * vContext
    contextGradient = weight * innerCost * vMain
    mainGradientBias = weight * innerCost
    contextGradientBias = weight * innerCost

    #Perform adaptive gradient descent for word vectors
    vMain -= (rateOfLearning * mainGradient / np.sqrt(gradWMain))
    vContext -= (rateOfLearning * contextGradient / np.sqrt(gradWContext))
    #Perform adaptive gradient descent for bias
    bMain -= (rateOfLearning * mainGradientBias / np.sqrt(gradBMain))
    bContext -= (rateOfLearning * contextGradientBias / np.sqrt(gradBContext))

    #Update squared gradient sums
    gradWMain += np.square(mainGradient)
    gradWContext += np.square(contextGradient)
    gradBMain += mainGradientBias ** 2
    gradBContext += contextGradientBias ** 2

  return costGlobal

###### Utility function for training the custom glove model

In [ ]:
#trains the glove MOdel with given vocabulary, coocurrence matrix, iterations and other parametrs.
#Returns the computed word vector matrix W of size 2V * D
def trainGloveModel(vocab, cooccurrences, numberOfDimensions, iterations, rateOfLearning, xMax, alpha):
  
  lengthOfVocab = len(vocab)
  W = (np.random.rand(lengthOfVocab * 2, numberOfDimensions) - 0.5) / float(numberOfDimensions + 1)
  biases = (np.random.rand(lengthOfVocab * 2) - 0.5) / float(numberOfDimensions + 1)
  gradient = np.ones((lengthOfVocab * 2, numberOfDimensions), dtype=np.float64)
  gradientBiases = np.ones(lengthOfVocab * 2, dtype=np.float64)

  #Building the data
  data = [ (W[iMain], W[iContext + lengthOfVocab],
            biases[iMain : iMain + 1],
            biases[iContext + lengthOfVocab : iContext + lengthOfVocab + 1],
            gradient[iMain], gradient[iContext + lengthOfVocab],
            gradientBiases[iMain : iMain + 1],
            gradientBiases[iContext + lengthOfVocab : iContext + lengthOfVocab + 1],
            cooccurrence )
            for iMain, iContext, cooccurrence in cooccurrences]

  #Training the word vector matrix for given number of iterations
  for i in tqdm(range(iterations)):
    cost = performIteration(vocab, data, rateOfLearning, xMax, alpha)
    print('Iteration', i, '- Cost:', cost)

  #Return the word vector matrix
  return W

###### Run entire workflow

In [ ]:
# utility function for running the entire workflow
def customGloveEmbeddings(dataset, contextWindow, numberOfDimensions, iterations, rateOfLearning, xMax, alpha):
  vocab, listOfSentences = constructVocab(dataset)
  cooccurrences = constructCoOccurenceMatrix(vocab, listOfSentences, contextWindow)
  W = trainGloveModel(vocab, cooccurrences, numberOfDimensions, iterations, rateOfLearning, xMax, alpha)
  return W, vocab

In [ ]:
W, vocab = customGloveEmbeddings(englishText, contextWindow=CONTEXT_WINDOW, numberOfDimensions=DIM_SIZE,
                            iterations=ITERATIONS, rateOfLearning=LEARNING_RATE,
                            xMax=X_MAX, alpha=ALPHA)

print('vocab Size:', len(vocab))

  2%|▏         | 1/45 [03:42<2:42:51, 222.08s/it]

Iteration 0 - Cost: 180729.55594677987


  4%|▍         | 2/45 [07:26<2:39:45, 222.92s/it]

Iteration 1 - Cost: 111847.27206446821


  7%|▋         | 3/45 [11:09<2:35:53, 222.70s/it]

...

 98%|█████████▊| 44/45 [2:42:14<03:40, 220.96s/it]

Iteration 43 - Cost: 31106.852705025543


100%|██████████| 45/45 [2:45:56<00:00, 221.26s/it]

Iteration 44 - Cost: 30717.776303014896


vocab Size: 76825


###### Save custom glove embeddings

In [ ]:
# Final GloVe word embedding is obtained by adding the center word embedding and context word embedding for each word
print(W.shape)
embeddings = {}
lengthOfVocab = len(vocab)
for word, (tokenID, _) in vocab.items():
  embeddings[word] = W[tokenID]+W[tokenID+lengthOfVocab]

# Save the embeddings in a pickle file
with open('embeddings.pickle', 'wb') as handle:
    pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save weights in a pickle file
with open('weights.pickle', 'wb') as handle:
    pickle.dump(W, handle, protocol=pickle.HIGHEST_PROTOCOL)

(153650, 100)


#### 2.2 - Comparing Word Similarity

###### Load custom glove embeddings

In [ ]:
# load embeddings implemented from the saved file
with open('embeddings.pickle', 'rb') as handle:
    embeddings = pickle.load(handle)

###### Sample test benchmarks

In [ ]:
# fetching multiple test benchamrks available
testBenchmarks = {
    "RG65": fetch_RG65(),
    "MTurk": fetch_MTurk(),
    "RW": fetch_RW(),
    "WS353": fetch_WS353(),
    "MEN": fetch_MEN(),
    "SIMLEX999": fetch_SimLex999(),
}

In [ ]:
# print some sample test cases of each benchmark
for name, data in testBenchmarks.items():
    print("Sample data from {}: pair \"{}\" and \"{}\" is assigned score {}".format(name, data.X[0][0], data.X[0][1], data.y[0]))

Sample data from RG65: pair "gem" and "jewel" is assigned score 9.85
Sample data from MTurk: pair "episcopal" and "russia" is assigned score 5.5
Sample data from RW: pair "squishing" and "squirt" is assigned score 5.88
Sample data from WS353: pair "love" and "sex" is assigned score 6.77
Sample data from MEN: pair "sun" and "sunlight" is assigned score [10.]
Sample data from SIMLEX999: pair "old" and "new" is assigned score 1.58


###### Performance evaluation on custom glove embeddings

In [ ]:
# calls inbuilt evaluate similarity function to evaluate performance of glove embeddings implemented
for name, data in testBenchmarks.items():
    print("Spearman correlation of scores on {} {}".format(name, evaluate_similarity(embeddings, data.X, data.y)))

Missing 30 words. Will replace them with mean vector


Spearman correlation of scores on RG65 0.1261666922886971


Missing 125 words. Will replace them with mean vector


Spearman correlation of scores on MTurk 0.17142783362215339


Missing 1743 words. Will replace them with mean vector


...

Spearman correlation of scores on WS353 0.19001707210564006


Missing 803 words. Will replace them with mean vector


Spearman correlation of scores on MEN 0.12444210895087617


Missing 94 words. Will replace them with mean vector


Spearman correlation of scores on SIMLEX999 0.06878120834230247


###### Load pre-trained glove embeddings

In [ ]:
# Fetch pre trained glove embeddings
gloveEmbeddings = fetch_GloVe(corpus="twitter-27B", dim=100)

File already downloaded, skipping


###### Performance evaluation on pre-trained embeddings

In [ ]:
# calls inbuilt evaluate similarity function to evaluate performance of pre-trained embeddings
for name, data in testBenchmarks.items():
    print("Spearman correlation of scores on {} {}".format(name, evaluate_similarity(gloveEmbeddings, data.X, data.y)))

Missing 1 words. Will replace them with mean vector
Missing 1071 words. Will replace them with mean vector
Missing 25 words. Will replace them with mean vector


Spearman correlation of scores on RG65 0.6774486160113895
Spearman correlation of scores on MTurk 0.5641004632190647
Spearman correlation of scores on RW 0.23074174522387267


Missing 1 words. Will replace them with mean vector
Missing 1 words. Will replace them with mean vector


Spearman correlation of scores on WS353 0.46979381939437287
Spearman correlation of scores on MEN 0.5773369776281995
Spearman correlation of scores on SIMLEX999 0.12221121100798378
